# Purpose: To compare the accuracy levels of a model trained with and without the augmented data set
- Steps:
    1. Build/reuse a LSTM based classification model, output: 1 out of 3 classes for each test sample.
    2. Use original dataset, train and test the accuracy level.
    3. Use augmented dataset (augmented rare classes + 80% of original data) for training, use 20% of original data for testing.
    4. Compare accuracy levels of 3 and 4. 

# 0. Initialization

In [13]:
########
# Imports
########
import tensorflow as tf
import keras
import os
import pandas as pd
import src.mutils as util

In [14]:
########
# Check for GPU
########
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# 1. Train the LSTM classification model w/ original dataset

In [11]:
df = pd.read_csv("./ScooterData/Front 1.mp4.csv") # Dataframe from Nth csv file

(array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([[ 0.00000000e+00,  9.08007920e-02, -6.12982869e-01, ...,
         -1.10081643e-01, -1.71017200e-02, -7.83752091e-03],
        [ 1.00000000e+00,  9.09924135e-02, -6.13051951e-01, ...,
         -1.10074483e-01, -1.73101872e-02, -7.88121857e-03],
        [ 2.00000000e+00,  9.09468457e-02, -6.13150060e-01, ...,
         -1.10040940e-01, -1.73895899e-02, -7.42972549e-03],
        ...,
        [ 4.21300000e+03,  1.68868661e-01, -5.80009401e-01, ...,
         -1.10891759e-01, -2.62079947e-02, -3.10749505e-02],
        [ 4.21400000e+03,  1.66271180e-01, -5.84010363e-01, ...,
         -1.11008853e-01, -2.50091739e-02, -3.10690124e-02],
        [ 4.21500000e+03,  1.64620250e-01, -5.84033012e-01, ...,
         -1.11168511e-01, -2.44276859e-02, -3.07505187e-02]]),
 array(['Stable', 'Stable', 'Stable', ..., 'Minimum Sway', 'Minimum Sway',
        'Minimum Sway'], dtype='<U12'))